# AD9833 Modulators test with MicroPython Remote
- https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf
- https://pypi.org/project/jupyter-micropython-remote/
- https://towardsdatascience.com/micropython-on-esp-using-jupyter-6f366ff5ed9

In [ ]:
%local
!ls /dev/ttyUSB*

In [1]:
# %connect /dev/ttyUSB0 --baudrate=115200 --user='micro' --password='python' --wait=0

%connect COM4 --baudrate=115200 --user='micro' --password='python' --wait=0

Connected on COM4


In [1]:
# %lsmagic

In [1]:
import peripherals
from ad9833 import AD9833

import machine
import gc

gc.collect()

## Debug mode?

In [1]:
AD9833.DEBUG_MODE_SHOW_BUS_DATA = False        # whether to show bus data. 
AD9833.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [1]:
with_hardware_device = True

if with_hardware_device:
    _spi = peripherals.SPI.get_uPy_spi(polarity = 1)
    _ss1 = peripherals.Pin.get_uPy_pin(pin_id = 15, output = True) 
    _ss2 = peripherals.Pin.get_uPy_pin(pin_id =  4, output = True) 
else:
    _spi = _ss1 = _ss2 = None  # using None for testing without actual hardware device.

ad1 = AD9833(_spi, _ss1)
ad2 = AD9833(_spi, _ss2)

ad1.reset()
ad2.reset()

In [1]:
bfsk = modulators.BFSK(ad1)
bpsk = modulators.BPSK(ad1)
dtmf = modulators.DTMF((ad1, ad2))
fm = modulators.FM(ad1)
iq = modulators.IQ((ad1, ad2))
ook = modulators.OOK(ad1)
pm = modulators.PM(ad1)
pwm = modulators.PWM(ad1)
qpsk = modulators.QPSK(ad1)

In [1]:
ad1.reset()
ad2.reset()

In [1]:
ad1.enable_output(False)
ad2.enable_output(False)

## Testing data sequence

In [1]:
%local
import numpy as np

samples_size = 30
duration = 0.1

symbols = np.random.randint(2, size = samples_size)
digital_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.uniform(-1, 1, size = samples_size).round(5)
analog_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice(['1', '2', '3', 'A', '4', '5', '6', 'B', '7', '8', '9', 'C', '*', '0', '#', 'D'], 
                           size = samples_size)
dtmf_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice([0, 1, 2, 3], size = samples_size)
quadrature_sequence = [(symbol, duration) for symbol in symbols] 

In [2]:
%local

print('digital_sequence = {}'.format(digital_sequence))
# print('analog_sequence = {}'.format(analog_sequence))
# print('dtmf_sequence = {}'.format(dtmf_sequence))
# print('quadrature_sequence = {}'.format(quadrature_sequence))

digital_sequence = [(0, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (1, 0.1)]


In [3]:
digital_sequence = [(1, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (0, 0.1), (0, 0.1), (0, 0.1), (1, 0.1), (1, 0.1), (0, 0.1), (1, 0.1), (0, 0.1), (0, 0.1)]

## Modulators test

In [3]:
bfsk.send_sequence(digital_sequence)

In [3]:
bpsk.send_sequence(digital_sequence)

In [3]:
ook.send_sequence(digital_sequence)

In [ ]:
fm.send_sequence(analog_sequence)

In [ ]:
pm.send_sequence(analog_sequence)

In [ ]:
qpsk.send_sequence(quadrature_sequence)

In [ ]:
iq.send_sequence(quadrature_sequence)

In [ ]:
dtmf.send_sequence(dtmf_sequence)

In [ ]:
pwm.run()